# Lezione 32 — Sentiment Analysis

## BLOCCO 4: Artificial Intelligence & NLP

---

## Obiettivi di Apprendimento

Al termine di questa lezione sarai in grado di:

1. **Definire** cosa si intende per Sentiment Analysis e le sue applicazioni pratiche
2. **Distinguere** tra approcci rule-based e approcci machine learning
3. **Implementare** un classificatore di sentiment con sklearn
4. **Valutare** le performance con metriche appropriate per testo
5. **Gestire** le sfide pratiche: negazione, sarcasmo, dominio-specificità

---

## Prerequisiti

- Lezione 30: Rappresentazione del testo (BoW, CountVectorizer)
- Lezione 31: TF-IDF e Text Mining
- Lezione 5-6: Classificazione supervisionata

---

## Indice

1. Teoria — Cos'è la Sentiment Analysis
2. Approcci e Sfide
3. Schema Mentale
4. Notebook Dimostrativo
5. Esercizi Svolti
6. Conclusione e Bignami

---

# 1. Teoria — Cos'è la Sentiment Analysis

## 1.1 Definizione Formale

La **Sentiment Analysis** (o Opinion Mining) è il task di NLP che consiste nell'identificare e classificare l'orientamento emotivo o valutativo espresso in un testo.

```
Input: "Questo prodotto è eccellente, lo consiglio vivamente!"
Output: POSITIVO (confidence: 0.92)

Input: "Pessima esperienza, mai più."
Output: NEGATIVO (confidence: 0.95)
```

## 1.2 Livelli di Analisi

| Livello | Granularità | Esempio |
|---------|-------------|---------|
| **Document-level** | Intero documento | Recensione: Positiva/Negativa |
| **Sentence-level** | Singola frase | "Il cibo era buono, ma il servizio pessimo" |
| **Aspect-level** | Caratteristica specifica | "Batteria: 4/5, Schermo: 2/5" |
| **Entity-level** | Entità menzionata | "Apple: positivo, Samsung: neutro" |

## 1.3 Schemi di Classificazione

**Schema Binario:**
- Positivo / Negativo

**Schema Ternario:**
- Positivo / Neutro / Negativo

**Schema Continuo:**
- Score da -1 (molto negativo) a +1 (molto positivo)

**Schema Multi-classe:**
- Molto negativo / Negativo / Neutro / Positivo / Molto positivo

## 1.4 Applicazioni nel Business

| Dominio | Applicazione | Valore |
|---------|--------------|--------|
| E-commerce | Analisi recensioni prodotti | Product improvement |
| Social Media | Brand monitoring | Reputation management |
| Finance | News sentiment per trading | Investment signals |
| Customer Service | Prioritizzazione ticket | Efficiency |
| HR | Analisi survey dipendenti | Employee satisfaction |

---

## 1.5 Approcci alla Sentiment Analysis

### Approccio 1: Rule-Based (Lexicon-Based)

Utilizza dizionari di parole con polarità predefinita.

**Come funziona:**
1. Tokenizza il testo
2. Cerca ogni token nel lexicon
3. Somma gli score di polarità
4. Classifica in base allo score totale

```
Lexicon esempio:
  "eccellente": +3
  "buono": +2
  "pessimo": -3
  "terribile": -4
  
Testo: "Prodotto eccellente ma prezzo pessimo"
Score: +3 + (-3) = 0 → Neutro
```

**Pro:**
- Interpretabile
- Non richiede dati di training
- Funziona out-of-the-box

**Contro:**
- Non cattura il contesto
- Problemi con negazione ("non è buono")
- Dominio-dipendente
- Ignora sarcasmo

### Approccio 2: Machine Learning Supervisionato

Addestra un classificatore su dati etichettati.

**Pipeline tipica:**
```
Testi etichettati → Preprocessing → Vectorization → Classificatore → Predizione
                                   (TF-IDF)        (Naive Bayes,
                                                    Logistic Reg,
                                                    SVM)
```

**Pro:**
- Cattura pattern complessi
- Adattabile al dominio
- Performance superiori con dati sufficienti

**Contro:**
- Richiede dati etichettati
- Meno interpretabile
- Richiede ritraining per nuovi domini

---

## 1.6 Le Sfide della Sentiment Analysis

### Sfida 1: Negazione

```
"Il film non è male" → Positivo (doppia negazione)
"Non mi piace affatto" → Negativo
"Non posso non consigliarlo" → Positivo
```

**Tecniche di mitigazione:**
- Prefisso "NOT_" ai token dopo negazione
- Finestre di negazione (3-5 token)
- Modelli che capiscono contesto (LSTM, Transformers)

### Sfida 2: Sarcasmo e Ironia

```
"Fantastico, un'altra giornata di pioggia" → Negativo (sarcasmo)
"Che bella sorpresa, il treno è in ritardo" → Negativo (ironia)
```

Approcci base non catturano queste sfumature.

### Sfida 3: Comparazioni

```
"Meglio di Samsung ma peggio di Apple" → Dipende dal target
"Non male come gli altri" → Relativamente positivo
```

### Sfida 4: Dominio-specificità

```
Recensione film: "La trama è prevedibile" → Negativo
Recensione software: "Il comportamento è prevedibile" → Positivo
```

Lo stesso termine ha polarità diverse in contesti diversi.

### Sfida 5: Soggettività vs Oggettività

```
"Il telefono ha 128GB di memoria" → Oggettivo (no sentiment)
"Il telefono ha troppa poca memoria" → Soggettivo (negativo)
```

Distinguere fatti da opinioni è un pre-task importante.

---

## 1.7 Metriche per Sentiment Analysis

### Quando le classi sono bilanciate

**Accuracy** è sufficiente:
$$Accuracy = \frac{Predizioni\ Corrette}{Totale\ Predizioni}$$

### Quando le classi sono sbilanciate (comune)

**Precision per classe:**
$$Precision_c = \frac{TP_c}{TP_c + FP_c}$$

**Recall per classe:**
$$Recall_c = \frac{TP_c}{TP_c + FN_c}$$

**F1-Score (media armonica):**
$$F1_c = 2 \times \frac{Precision_c \times Recall_c}{Precision_c + Recall_c}$$

### Aggregazione Multi-classe

| Metodo | Formula | Uso |
|--------|---------|-----|
| **Macro** | Media semplice per classe | Tutte le classi ugualmente importanti |
| **Weighted** | Media pesata per supporto | Classi sbilanciate |
| **Micro** | Aggregazione globale | Focus su accuracy generale |

### Interpretazione Pratica

| Obiettivo | Priorità |
|-----------|----------|
| Evitare falsi positivi | Alta Precision |
| Evitare falsi negativi | Alta Recall |
| Bilanciare entrambi | Alto F1-Score |

Per il sentiment: spesso preferiamo alta **recall sulla classe negativa** (non perdere clienti insoddisfatti).

---

# 2. Schema Mentale — Mappa Logica

## Pipeline Completa Sentiment Analysis

```
                    SENTIMENT ANALYSIS PIPELINE
                    
┌─────────────────────────────────────────────────────────────┐
│                      INPUT: Testo Raw                       │
│              "Prodotto fantastico, ma spedizione lenta"     │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                    PREPROCESSING                            │
│  • Lowercase                                                │
│  • Rimozione punteggiatura                                  │
│  • Tokenizzazione                                           │
│  • Rimozione stopwords (opzionale)                          │
│  • Gestione negazione (opzionale)                           │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                   VECTORIZATION                             │
│                                                             │
│    Bag of Words         TF-IDF          N-grams            │
│    (CountVectorizer)    (TfidfVec)      (ngram_range)      │
│                                                             │
│    Matrice sparsa: (n_docs, vocab_size)                     │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                    CLASSIFICATORE                           │
│                                                             │
│   ┌───────────┐  ┌───────────┐  ┌───────────┐              │
│   │   Naive   │  │ Logistic  │  │    SVM    │              │
│   │   Bayes   │  │ Regression│  │  (Linear) │              │
│   └───────────┘  └───────────┘  └───────────┘              │
│                                                             │
│   Baseline       Interpretabile   Alte performance          │
│   Veloce         Probabilità      Margini                  │
└─────────────────────────────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────┐
│                     OUTPUT                                  │
│           Classe: POSITIVO  |  Probabilità: 0.78            │
│           (o score continuo: +0.56)                         │
└─────────────────────────────────────────────────────────────┘
```

## Scelta del Classificatore

```
                    QUALE CLASSIFICATORE?
                    
              Velocità importante?
                     │
         ┌───────────┴───────────┐
         │ Sì                    │ No
         ▼                       ▼
    Naive Bayes            Dataset grande?
    (baseline)                   │
                     ┌───────────┴───────────┐
                     │ Sì                    │ No
                     ▼                       ▼
                   SVM               Logistic Regression
              (LinearSVC)              (interpretabile)
```

## Checklist Operativa

```
□ Dati etichettati sufficienti (>1000 per classe)
□ Classi bilanciate (o stratificazione)
□ Preprocessing coerente train/test
□ Vectorizer fittato SOLO su train
□ Cross-validation per tuning
□ Metriche appropriate (F1 se sbilanciato)
□ Analisi errori (confusion matrix)
```

---

# 3. Notebook Dimostrativo

## Setup e Dati di Esempio

In [ ]:
# ============================================================
# DEMO 1: Creazione Dataset di Recensioni
# ============================================================

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import (accuracy_score, classification_report, 
                             confusion_matrix, f1_score)
import warnings
warnings.filterwarnings('ignore')

# Dataset di recensioni in italiano (simulato ma realistico)
recensioni = [
    # Positive
    ("Prodotto eccellente, super consigliato!", "positivo"),
    ("Ottima qualità, sono molto soddisfatto", "positivo"),
    ("Spedizione veloce e prodotto conforme", "positivo"),
    ("Il migliore acquisto che abbia mai fatto", "positivo"),
    ("Fantastico, lo ricomprerei subito", "positivo"),
    ("Qualità prezzo imbattibile", "positivo"),
    ("Funziona perfettamente, molto contento", "positivo"),
    ("Servizio clienti impeccabile", "positivo"),
    ("Bellissimo, esattamente come in foto", "positivo"),
    ("Top, consiglio vivamente a tutti", "positivo"),
    ("Arrivo puntuale, qualità ottima", "positivo"),
    ("Sono rimasto piacevolmente sorpreso", "positivo"),
    ("Rapporto qualità prezzo eccezionale", "positivo"),
    ("Prodotto robusto e ben fatto", "positivo"),
    ("Supera le aspettative", "positivo"),
    
    # Negative
    ("Prodotto scadente, non lo consiglio", "negativo"),
    ("Pessima qualità, si è rotto subito", "negativo"),
    ("Spedizione lentissima, mai più", "negativo"),
    ("Deluso totalmente, soldi buttati", "negativo"),
    ("Non funziona come descritto", "negativo"),
    ("Qualità scarsa, materiali economici", "negativo"),
    ("Servizio clienti inesistente", "negativo"),
    ("Prodotto difettoso, reso immediato", "negativo"),
    ("Non vale assolutamente il prezzo", "negativo"),
    ("Esperienza terribile, da evitare", "negativo"),
    ("Arrivato danneggiato, vergognoso", "negativo"),
    ("Totale delusione, non corrisponde", "negativo"),
    ("Materiale scadente, si rompe facilmente", "negativo"),
    ("Mai più, assistenza pessima", "negativo"),
    ("Prodotto inutile, non lo ricompro", "negativo"),
]

# Creazione DataFrame
# Ogni riga è una recensione con la sua etichetta
df = pd.DataFrame(recensioni, columns=['testo', 'sentiment'])

print("="*60)
print("DATASET RECENSIONI")
print("="*60)
print(f"\nDimensione dataset: {len(df)} recensioni")
print(f"\nDistribuzione classi:")
print(df['sentiment'].value_counts())
print(f"\nEsempi di recensioni:")
print(df.head(10).to_string(index=False))

In [ ]:
# ============================================================
# DEMO 2: Approccio Rule-Based (Lexicon)
# ============================================================
# Implementazione semplice di sentiment lexicon-based
# per capire il concetto prima del ML

# Dizionario di polarità italiano (semplificato)
lexicon = {
    # Parole positive
    'eccellente': 3, 'ottimo': 2, 'ottima': 2, 'fantastico': 3,
    'consigliato': 2, 'soddisfatto': 2, 'contento': 2, 'bellissimo': 3,
    'perfettamente': 2, 'impeccabile': 3, 'top': 2, 'sorpreso': 1,
    'migliore': 2, 'robusto': 1, 'veloce': 1, 'puntuale': 1,
    'eccezionale': 3, 'imbattibile': 2, 'vivamente': 1,
    
    # Parole negative
    'scadente': -2, 'pessima': -3, 'pessimo': -3, 'lentissima': -2,
    'deluso': -2, 'delusione': -2, 'buttati': -2, 'terribile': -3,
    'difettoso': -2, 'vergognoso': -3, 'inesistente': -2,
    'scarsa': -2, 'economici': -1, 'rotto': -2, 'danneggiato': -2,
    'inutile': -2, 'evitare': -2,
}

def sentiment_lexicon(testo, lexicon):
    """
    Calcola sentiment score basato su lexicon.
    
    Args:
        testo: stringa da analizzare
        lexicon: dizionario {parola: score}
    
    Returns:
        tuple: (score_totale, classe_predetta, parole_trovate)
    """
    # Tokenizzazione semplice: lowercase e split
    tokens = testo.lower().split()
    
    # Calcolo score
    score = 0
    parole_trovate = []
    
    for token in tokens:
        # Rimuovi punteggiatura dal token
        token_pulito = ''.join(c for c in token if c.isalnum())
        
        if token_pulito in lexicon:
            score += lexicon[token_pulito]
            parole_trovate.append((token_pulito, lexicon[token_pulito]))
    
    # Classificazione basata sullo score
    if score > 0:
        classe = 'positivo'
    elif score < 0:
        classe = 'negativo'
    else:
        classe = 'neutro'
    
    return score, classe, parole_trovate

# Test su alcune recensioni
print("="*60)
print("APPROCCIO LEXICON-BASED")
print("="*60)

test_reviews = [
    "Prodotto eccellente, super consigliato!",
    "Pessima qualità, si è rotto subito",
    "Prodotto normale, niente di speciale",  # Neutro
    "Non è male, ma nemmeno eccezionale",    # Difficile
]

for review in test_reviews:
    score, classe, parole = sentiment_lexicon(review, lexicon)
    print(f"\nTesto: '{review}'")
    print(f"  Score: {score:+d} → Classe: {classe.upper()}")
    print(f"  Parole rilevate: {parole}")

In [ ]:
# ============================================================
# DEMO 3: Pipeline ML con Naive Bayes
# ============================================================
# Approccio supervisionato: addestramento su dati etichettati

# Separazione testi e labels
X = df['testo'].values  # Feature: testi
y = df['sentiment'].values  # Target: etichette

# Split train/test con stratificazione
# stratify=y mantiene proporzione classi in entrambi i set
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.3,  # 30% per test
    random_state=42,
    stratify=y  # Mantiene proporzione positivo/negativo
)

print("="*60)
print("SPLIT TRAIN/TEST")
print("="*60)
print(f"Training set: {len(X_train)} documenti")
print(f"Test set: {len(X_test)} documenti")

# Verifica bilanciamento dopo split
from collections import Counter
print(f"\nDistribuzione training: {Counter(y_train)}")
print(f"Distribuzione test: {Counter(y_test)}")

# Step 1: Vectorization con TF-IDF
# fit_transform su train, solo transform su test
vectorizer = TfidfVectorizer(
    max_features=1000,  # Limita vocabolario
    ngram_range=(1, 2),  # Unigram e bigram
    min_df=1,  # Minimo 1 documento (dataset piccolo)
)

# IMPORTANTE: fit SOLO sul training set
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)  # Solo transform!

print(f"\nDimensioni matrice TF-IDF:")
print(f"  Training: {X_train_tfidf.shape}")
print(f"  Test: {X_test_tfidf.shape}")

# Step 2: Addestramento Naive Bayes
# Multinomial NB è ideale per dati di testo (counts/frequencies)
nb_classifier = MultinomialNB()
nb_classifier.fit(X_train_tfidf, y_train)

# Step 3: Predizione e Valutazione
y_pred = nb_classifier.predict(X_test_tfidf)

print(f"\n{'='*60}")
print("RISULTATI NAIVE BAYES")
print("="*60)
print(f"Accuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# ============================================================
# DEMO 4: Confronto Classificatori
# ============================================================
# Testiamo più classificatori per vedere quale performa meglio

# Dizionario classificatori da testare
classificatori = {
    'Naive Bayes': MultinomialNB(),
    'Logistic Regression': LogisticRegression(max_iter=1000, random_state=42),
    'Linear SVM': LinearSVC(random_state=42, max_iter=1000),
}

print("="*60)
print("CONFRONTO CLASSIFICATORI")
print("="*60)

risultati = []

for nome, clf in classificatori.items():
    # Addestramento
    clf.fit(X_train_tfidf, y_train)
    
    # Predizione
    y_pred = clf.predict(X_test_tfidf)
    
    # Metriche
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, pos_label='positivo')
    
    risultati.append({
        'Classificatore': nome,
        'Accuracy': f"{acc:.2%}",
        'F1-Score': f"{f1:.2%}"
    })
    
    print(f"\n{nome}:")
    print(f"  Accuracy: {acc:.2%}")
    print(f"  F1 (positivo): {f1:.2%}")

# Tabella riassuntiva
df_risultati = pd.DataFrame(risultati)
print(f"\n{'='*60}")
print("TABELLA RIASSUNTIVA")
print("="*60)
print(df_risultati.to_string(index=False))

In [ ]:
# ============================================================
# DEMO 5: Confusion Matrix e Analisi Errori
# ============================================================
# Visualizziamo dove il modello sbaglia

# Usiamo Logistic Regression come modello finale
clf_finale = LogisticRegression(max_iter=1000, random_state=42)
clf_finale.fit(X_train_tfidf, y_train)
y_pred_finale = clf_finale.predict(X_test_tfidf)

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred_finale, labels=['negativo', 'positivo'])

print("="*60)
print("CONFUSION MATRIX")
print("="*60)
print(f"""
                    PREDETTO
                  Neg    Pos
            ┌─────────┬─────────┐
REALE  Neg  │   {cm[0,0]:^3}   │   {cm[0,1]:^3}   │
            ├─────────┼─────────┤
       Pos  │   {cm[1,0]:^3}   │   {cm[1,1]:^3}   │
            └─────────┴─────────┘
""")

# Interpretazione
print("Interpretazione:")
print(f"  • True Negatives (Neg→Neg): {cm[0,0]}")
print(f"  • False Positives (Neg→Pos): {cm[0,1]} - Tipo I error")
print(f"  • False Negatives (Pos→Neg): {cm[1,0]} - Tipo II error")
print(f"  • True Positives (Pos→Pos): {cm[1,1]}")

# Analisi errori: quali testi sono stati classificati male?
print(f"\n{'='*60}")
print("ANALISI ERRORI")
print("="*60)

errori = []
for i, (testo, pred, reale) in enumerate(zip(X_test, y_pred_finale, y_test)):
    if pred != reale:
        errori.append({
            'Testo': testo[:50] + '...' if len(testo) > 50 else testo,
            'Reale': reale,
            'Predetto': pred
        })

if errori:
    print("\nTesti classificati erroneamente:")
    for e in errori:
        print(f"\n  '{e['Testo']}'")
        print(f"    Reale: {e['Reale']} | Predetto: {e['Predetto']}")
else:
    print("\nNessun errore nel test set!")

In [ ]:
# ============================================================
# DEMO 6: Interpretabilità - Parole più Importanti
# ============================================================
# Quali parole influenzano di più la classificazione?

# Logistic Regression ha coefficienti interpretabili
feature_names = vectorizer.get_feature_names_out()
coefficients = clf_finale.coef_[0]  # Coefficienti del modello

# Creiamo DataFrame con feature e coefficienti
importance_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': coefficients
})

# Ordiniamo per coefficiente
# Coefficienti positivi → favoriscono classe "positivo"
# Coefficienti negativi → favoriscono classe "negativo"

print("="*60)
print("PAROLE PIÙ INDICATIVE PER SENTIMENT")
print("="*60)

# Top 10 parole per sentiment POSITIVO
top_positive = importance_df.nlargest(10, 'coefficient')
print("\n📈 Top 10 parole → POSITIVO:")
for _, row in top_positive.iterrows():
    print(f"  {row['feature']:25} (coef: {row['coefficient']:+.3f})")

# Top 10 parole per sentiment NEGATIVO
top_negative = importance_df.nsmallest(10, 'coefficient')
print("\n📉 Top 10 parole → NEGATIVO:")
for _, row in top_negative.iterrows():
    print(f"  {row['feature']:25} (coef: {row['coefficient']:+.3f})")

# Insight operativo
print(f"\n{'='*60}")
print("INSIGHT")
print("="*60)
print("""
Il modello ha imparato automaticamente quali parole indicano
sentiment positivo o negativo, senza bisogno di un lexicon manuale.

I coefficienti mostrano l'importanza relativa di ogni termine:
  • Coefficiente alto positivo → forte indicatore positivo
  • Coefficiente alto negativo → forte indicatore negativo
  • Coefficiente vicino a 0 → parola neutrale
""")

In [ ]:
# ============================================================
# DEMO 7: Predizione su Nuovi Testi
# ============================================================
# Come usare il modello addestrato su testi nuovi

def predici_sentiment(testo, vectorizer, classifier):
    """
    Predice il sentiment di un nuovo testo.
    
    Args:
        testo: stringa o lista di stringhe
        vectorizer: TfidfVectorizer già fittato
        classifier: classificatore già addestrato
    
    Returns:
        tuple: (predizione, probabilità) se disponibile
    """
    # Se è una singola stringa, convertila in lista
    if isinstance(testo, str):
        testo = [testo]
    
    # Trasforma il testo (NON fit!)
    X_new = vectorizer.transform(testo)
    
    # Predizione
    predizioni = classifier.predict(X_new)
    
    # Probabilità (se disponibili)
    if hasattr(classifier, 'predict_proba'):
        probabilita = classifier.predict_proba(X_new)
        return list(zip(predizioni, probabilita.max(axis=1)))
    else:
        return list(zip(predizioni, [None]*len(predizioni)))

# Test su nuovi testi
nuovi_testi = [
    "Un prodotto davvero eccezionale, lo adoro!",
    "Mai più, esperienza pessima dall'inizio alla fine",
    "Normale, niente di che",
    "Il prezzo è giusto ma la qualità poteva essere migliore",
]

print("="*60)
print("PREDIZIONE SU NUOVI TESTI")
print("="*60)

for testo in nuovi_testi:
    risultato = predici_sentiment(testo, vectorizer, clf_finale)
    pred, prob = risultato[0]
    print(f"\nTesto: '{testo}'")
    print(f"  → Sentiment: {pred.upper()}")
    if prob is not None:
        print(f"  → Confidenza: {prob:.1%}")

In [ ]:
# ============================================================
# DEMO 8: Cross-Validation per Robustezza
# ============================================================
# La valutazione su un solo split può essere instabile
# Cross-validation dà una stima più robusta

from sklearn.model_selection import cross_val_score, StratifiedKFold

# Usiamo TUTTI i dati per cross-validation
X_all = vectorizer.fit_transform(df['testo'].values)
y_all = df['sentiment'].values

# Definizione strategia di split
# StratifiedKFold mantiene proporzione classi in ogni fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

print("="*60)
print("CROSS-VALIDATION (5-Fold)")
print("="*60)

for nome, clf in classificatori.items():
    # Cross-validation con accuracy
    scores = cross_val_score(clf, X_all, y_all, cv=cv, scoring='accuracy')
    
    print(f"\n{nome}:")
    print(f"  Scores per fold: {[f'{s:.2%}' for s in scores]}")
    print(f"  Media ± Std: {scores.mean():.2%} ± {scores.std():.2%}")

print(f"\n{'='*60}")
print("INTERPRETAZIONE")
print("="*60)
print("""
La cross-validation divide i dati in K fold e addestra K modelli,
ognuno usando K-1 fold per training e 1 per test.

• Media alta + Std bassa = Modello stabile e affidabile
• Media alta + Std alta = Modello instabile
• Std < 5% è generalmente accettabile

Con dataset piccoli, la varianza può essere alta.
""")

---

# 4. Esercizi Svolti

## Esercizio 1: Pipeline Completa di Sentiment Analysis

**Obiettivo:** Costruire una pipeline completa di sentiment analysis su un dataset più ampio, includendo preprocessing, training, valutazione e analisi degli errori.

In [ ]:
# ============================================================
# ESERCIZIO 1 - SOLUZIONE
# Pipeline Completa di Sentiment Analysis
# ============================================================

import re
from sklearn.pipeline import Pipeline

# Step 1: Dataset esteso (simula dati reali)
recensioni_estese = [
    # Positive (20)
    ("Prodotto fantastico, qualità eccezionale", "positivo"),
    ("Consegna rapidissima, tutto perfetto", "positivo"),
    ("Miglior acquisto dell'anno, super soddisfatto", "positivo"),
    ("Ottimo rapporto qualità prezzo", "positivo"),
    ("Funziona benissimo, come descritto", "positivo"),
    ("Materiali di prima scelta, molto robusto", "positivo"),
    ("Servizio clienti eccellente e disponibile", "positivo"),
    ("Prodotto superiore alle aspettative", "positivo"),
    ("Lo consiglio assolutamente a tutti", "positivo"),
    ("Bellissimo design, funzionale", "positivo"),
    ("Arrivo puntuale, packaging perfetto", "positivo"),
    ("Qualità top, prezzo giusto", "positivo"),
    ("Molto soddisfatto dell'acquisto", "positivo"),
    ("Funziona alla perfezione ogni volta", "positivo"),
    ("Prodotto robusto e ben costruito", "positivo"),
    ("Supera ogni aspettativa, consigliatissimo", "positivo"),
    ("Servizio impeccabile dall'inizio alla fine", "positivo"),
    ("Acquisto che rifarei subito", "positivo"),
    ("Ottima scelta, nessun rimpianto", "positivo"),
    ("Prodotto di qualità premium", "positivo"),
    
    # Negative (20)
    ("Prodotto scadente, si è rotto subito", "negativo"),
    ("Pessima qualità, non vale il prezzo", "negativo"),
    ("Spedizione lentissima, pacco danneggiato", "negativo"),
    ("Totale delusione, non corrisponde alle foto", "negativo"),
    ("Materiale economico, si graffia facilmente", "negativo"),
    ("Assistenza inesistente, nessuna risposta", "negativo"),
    ("Prodotto difettoso, devo renderlo", "negativo"),
    ("Mai più, esperienza terribile", "negativo"),
    ("Non funziona come promesso", "negativo"),
    ("Qualità pessima, soldi buttati", "negativo"),
    ("Arrivato in ritardo e rotto", "negativo"),
    ("Servizio clienti inutile e scortese", "negativo"),
    ("Prodotto fragile, durato una settimana", "negativo"),
    ("Non lo consiglio a nessuno", "negativo"),
    ("Peggiore acquisto della mia vita", "negativo"),
    ("Truffa totale, prodotto diverso", "negativo"),
    ("Qualità scadentissima, evitate", "negativo"),
    ("Reso immediato, inaccettabile", "negativo"),
    ("Prodotto inutilizzabile", "negativo"),
    ("Esperienza da dimenticare", "negativo"),
]

# Creazione DataFrame
df_ext = pd.DataFrame(recensioni_estese, columns=['testo', 'sentiment'])

# Step 2: Funzione di preprocessing
def preprocess_text(testo):
    """
    Preprocessing standard per testo italiano.
    
    Steps:
    1. Lowercase
    2. Rimozione caratteri speciali (mantiene lettere e spazi)
    3. Rimozione spazi multipli
    """
    # Lowercase
    testo = testo.lower()
    
    # Rimuovi caratteri speciali (mantieni lettere accentate italiane)
    testo = re.sub(r'[^a-zàèéìòù\s]', '', testo)
    
    # Rimuovi spazi multipli
    testo = re.sub(r'\s+', ' ', testo).strip()
    
    return testo

# Applica preprocessing
df_ext['testo_clean'] = df_ext['testo'].apply(preprocess_text)

print("="*60)
print("ESERCIZIO 1: PIPELINE COMPLETA")
print("="*60)
print(f"\nDataset: {len(df_ext)} documenti")
print(f"Distribuzione: {df_ext['sentiment'].value_counts().to_dict()}")
print(f"\nEsempio preprocessing:")
print(f"  Originale: '{df_ext.iloc[0]['testo']}'")
print(f"  Pulito:    '{df_ext.iloc[0]['testo_clean']}'")

# Step 3: Split con stratificazione
X = df_ext['testo_clean'].values
y = df_ext['sentiment'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Step 4: Pipeline sklearn
# Combina vectorizer e classificatore in un unico oggetto
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=500)),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

# Addestramento pipeline
pipeline.fit(X_train, y_train)

# Step 5: Valutazione
y_pred = pipeline.predict(X_test)

print(f"\n{'='*60}")
print("RISULTATI")
print("="*60)
print(f"\nAccuracy: {accuracy_score(y_test, y_pred):.2%}")
print(f"\nClassification Report:")
print(classification_report(y_test, y_pred))

# Step 6: Analisi errori
print(f"{'='*60}")
print("ANALISI ERRORI")
print("="*60)
errori_idx = np.where(y_pred != y_test)[0]
print(f"\nNumero errori: {len(errori_idx)} su {len(y_test)}")

for idx in errori_idx[:5]:  # Mostra max 5 errori
    print(f"\n  Testo: '{X_test[idx][:60]}...'")
    print(f"  Reale: {y_test[idx]} | Predetto: {y_pred[idx]}")

---

## Esercizio 2: Confronto BoW vs TF-IDF per Sentiment

**Obiettivo:** Confrontare le performance di Bag of Words e TF-IDF nella classificazione del sentiment, usando lo stesso classificatore.

In [ ]:
# ============================================================
# ESERCIZIO 2 - SOLUZIONE
# Confronto BoW vs TF-IDF per Sentiment
# ============================================================

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Uso lo stesso dataset dell'esercizio 1
X = df_ext['testo_clean'].values
y = df_ext['sentiment'].values

# Split (stesso seed per confronto equo)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Configurazioni da testare
configurazioni = {
    'BoW (unigram)': CountVectorizer(ngram_range=(1, 1)),
    'BoW (bigram)': CountVectorizer(ngram_range=(1, 2)),
    'TF-IDF (unigram)': TfidfVectorizer(ngram_range=(1, 1)),
    'TF-IDF (bigram)': TfidfVectorizer(ngram_range=(1, 2)),
}

print("="*60)
print("ESERCIZIO 2: CONFRONTO BoW vs TF-IDF")
print("="*60)

risultati = []

for nome, vectorizer in configurazioni.items():
    # Fit su training, transform su entrambi
    X_train_vec = vectorizer.fit_transform(X_train)
    X_test_vec = vectorizer.transform(X_test)
    
    # Stesso classificatore per tutti
    clf = LogisticRegression(max_iter=1000, random_state=42)
    clf.fit(X_train_vec, y_train)
    
    # Valutazione
    y_pred = clf.predict(X_test_vec)
    acc = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, pos_label='positivo')
    
    risultati.append({
        'Configurazione': nome,
        'Vocab Size': X_train_vec.shape[1],
        'Accuracy': f"{acc:.2%}",
        'F1-Score': f"{f1:.2%}"
    })
    
    print(f"\n{nome}:")
    print(f"  Dimensione vocabolario: {X_train_vec.shape[1]}")
    print(f"  Accuracy: {acc:.2%}")
    print(f"  F1 (positivo): {f1:.2%}")

# Tabella comparativa
print(f"\n{'='*60}")
print("TABELLA COMPARATIVA")
print("="*60)
df_confronto = pd.DataFrame(risultati)
print(df_confronto.to_string(index=False))

# Insight
print(f"\n{'='*60}")
print("INSIGHT")
print("="*60)
print("""
Osservazioni tipiche:
• TF-IDF spesso supera BoW perché penalizza parole troppo comuni
• I bigram catturano pattern come "non buono", "molto soddisfatto"
• Con dataset piccoli, le differenze possono essere minime
• TF-IDF + bigram è spesso la scelta migliore per sentiment
""")

---

## Esercizio 3: Sistema di Alert per Recensioni Negative

**Obiettivo:** Creare un sistema che identifica le recensioni più negative per prioritizzare l'intervento del customer service.

In [ ]:
# ============================================================
# ESERCIZIO 3 - SOLUZIONE
# Sistema di Alert per Recensioni Negative
# ============================================================

class SentimentAlertSystem:
    """
    Sistema di monitoraggio sentiment per customer service.
    
    Funzionalità:
    1. Classifica nuove recensioni
    2. Calcola score di negatività
    3. Prioritizza interventi basandosi su confidence
    """
    
    def __init__(self):
        # Pipeline TF-IDF + Logistic Regression
        self.pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=1000)),
            ('clf', LogisticRegression(max_iter=1000, random_state=42))
        ])
        self.is_trained = False
    
    def train(self, testi, labels):
        """Addestra il sistema su dati storici."""
        self.pipeline.fit(testi, labels)
        self.is_trained = True
        print(f"Sistema addestrato su {len(testi)} recensioni")
    
    def analyze(self, testi):
        """
        Analizza un batch di recensioni.
        
        Returns:
            DataFrame con: testo, sentiment, probabilità, priorità
        """
        if not self.is_trained:
            raise ValueError("Sistema non addestrato!")
        
        # Predizioni e probabilità
        predizioni = self.pipeline.predict(testi)
        probabilita = self.pipeline.predict_proba(testi)
        
        # Costruisci risultati
        risultati = []
        for testo, pred, probs in zip(testi, predizioni, probabilita):
            # prob[0] = negativo, prob[1] = positivo (ordine alfabetico)
            prob_negativo = probs[0]  # Probabilità classe negativa
            prob_positivo = probs[1]
            
            # Calcola priorità: alta se negativo E alta confidenza
            if pred == 'negativo':
                priorita = 'ALTA' if prob_negativo > 0.8 else 'MEDIA'
            else:
                priorita = 'BASSA'
            
            risultati.append({
                'testo': testo[:50] + '...' if len(testo) > 50 else testo,
                'sentiment': pred,
                'confidenza': max(prob_negativo, prob_positivo),
                'priorita': priorita
            })
        
        return pd.DataFrame(risultati)
    
    def get_alerts(self, testi, threshold=0.7):
        """
        Restituisce solo le recensioni che richiedono attenzione.
        
        Args:
            testi: lista di recensioni
            threshold: soglia minima di confidenza per alert
        """
        df_analisi = self.analyze(testi)
        
        # Filtra: negative con alta confidenza
        alerts = df_analisi[
            (df_analisi['sentiment'] == 'negativo') & 
            (df_analisi['confidenza'] >= threshold)
        ].sort_values('confidenza', ascending=False)
        
        return alerts

# Addestramento sistema
alert_system = SentimentAlertSystem()
alert_system.train(df_ext['testo_clean'].values, df_ext['sentiment'].values)

# Simula arrivo nuove recensioni
nuove_recensioni = [
    "Prodotto arrivato rotto, assistenza inesistente, voglio rimborso!",
    "Tutto ok, niente di speciale",
    "Fantastico acquisto, super contento",
    "Delusione totale, peggio di così non poteva andare",
    "Discreto, poteva essere migliore",
    "Servizio pessimo, non risponde nessuno, vergogna!",
    "Ottima qualità, lo consiglio",
]

print("="*60)
print("ESERCIZIO 3: SISTEMA ALERT")
print("="*60)

# Analisi completa
print("\n📊 ANALISI COMPLETA:")
df_analisi = alert_system.analyze(nuove_recensioni)
print(df_analisi.to_string(index=False))

# Solo alert
print(f"\n{'='*60}")
print("🚨 ALERT - Recensioni che richiedono intervento:")
print("="*60)
alerts = alert_system.get_alerts(nuove_recensioni, threshold=0.7)
if len(alerts) > 0:
    print(alerts.to_string(index=False))
else:
    print("Nessun alert critico")

# Report statistico
print(f"\n{'='*60}")
print("📈 REPORT STATISTICO")
print("="*60)
print(f"Totale recensioni analizzate: {len(nuove_recensioni)}")
print(f"Negative: {len(df_analisi[df_analisi['sentiment'] == 'negativo'])}")
print(f"Positive: {len(df_analisi[df_analisi['sentiment'] == 'positivo'])}")
print(f"Alert alta priorità: {len(df_analisi[df_analisi['priorita'] == 'ALTA'])}")

---

# 5. Conclusione Operativa

## Cosa Abbiamo Imparato

| Concetto | Definizione | Applicazione |
|----------|-------------|--------------|
| **Sentiment Analysis** | Classificazione di testi per polarità | Recensioni, social media, survey |
| **Lexicon-based** | Dizionari di parole con score | Prototipazione rapida, baseline |
| **ML-based** | Classificatori addestrati su dati | Produzione, alta accuratezza |
| **Preprocessing** | Pulizia e normalizzazione testo | Step obbligatorio pre-ML |
| **Confusion Matrix** | Matrice di errori | Diagnostica modello |

## Quando Usare Sentiment Analysis

| Caso d'Uso | Approccio Consigliato | Perché |
|------------|----------------------|--------|
| PoC veloce | Lexicon-based | Nessun training richiesto |
| Produzione | ML (LogReg + TF-IDF) | Bilanciato accuratezza/velocità |
| Alta accuratezza | Ensemble o Deep Learning | Cattura pattern complessi |
| Dominio specifico | Fine-tuning su dati dominio | Adatta lessico |

## Workflow Operativo

```python
# Template produzione
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

# 1. Prepara dati
X_train, X_test, y_train, y_test = train_test_split(...)

# 2. Costruisci pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2))),
    ('clf', LogisticRegression(max_iter=1000))
])

# 3. Valida con cross-validation
scores = cross_val_score(pipeline, X_train, y_train, cv=5)
print(f"CV Score: {scores.mean():.2%} ± {scores.std():.2%}")

# 4. Train finale e test
pipeline.fit(X_train, y_train)
print(classification_report(y_test, pipeline.predict(X_test)))

# 5. Deploy
joblib.dump(pipeline, 'sentiment_model.pkl')
```

## Errori Comuni da Evitare

1. **Data leakage**: fit vectorizer su tutto il dataset invece che solo train
2. **Ignorare sbilanciamento**: accuracy alta ma recall bassa su classe minoritaria
3. **No preprocessing**: rumore e inconsistenza nei dati
4. **Overfitting**: modello troppo complesso per dati scarsi
5. **Ignorare il dominio**: sentiment diverso in contesti diversi

---

# 6. Bignami — Scheda di Riferimento Rapido

## Pipeline Minima

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Costruzione
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=5000)),
    ('clf', LogisticRegression(max_iter=1000))
])

# Training
pipeline.fit(X_train, y_train)

# Predizione
sentiments = pipeline.predict(new_texts)
probabilities = pipeline.predict_proba(new_texts)
```

## Classificatori per Sentiment

| Classificatore | Import | Pro | Contro |
|----------------|--------|-----|--------|
| Naive Bayes | `MultinomialNB()` | Veloce, baseline | Assume indipendenza |
| Logistic Regression | `LogisticRegression()` | Interpretabile | Lineare |
| Linear SVM | `LinearSVC()` | Alte performance | No probabilità |

## Metriche Chiave

```python
from sklearn.metrics import (accuracy_score, f1_score,
                              classification_report, confusion_matrix)

# Report completo
print(classification_report(y_true, y_pred))

# Singole metriche
acc = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, pos_label='negativo')
cm = confusion_matrix(y_true, y_pred)
```

## Scelta Rapida Metriche

| Scenario | Metrica | Motivazione |
|----------|---------|-------------|
| Classi bilanciate | Accuracy | Semplice e interpretabile |
| Evitare falsi allarmi | Precision | Minimizza FP |
| Non perdere critici | Recall | Minimizza FN |
| Bilanciato | F1-Score | Media armonica |

## Checklist Pre-Deploy

```
□ Split stratificato
□ Vectorizer fittato solo su train
□ Cross-validation completata
□ Confusion matrix analizzata
□ Performance su classe minoritaria ok
□ Pipeline serializzata con joblib
```

---
*Fine Lezione 32 — Sentiment Analysis*